<a href="https://colab.research.google.com/github/brilliantB/Machine-Learning/blob/main/%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_surprise%EC%9D%B4%EC%9A%A9_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surprise 사용 

In [1]:
!pip install scikit-surprise

In [2]:
import surprise
print (surprise.__version__)

1.1.1


In [3]:
from surprise import SVD            # 고유값분해(행렬을 더 낮은 차원으로 분해)해서 다시 원본으로 살리면서 잠재먹인 값을 추론하는 방법
from surprise import Dataset        # SVD를 사용하기 위한 데이터셋 만들어주는 클래스 / 사용자, 아이템, 평점
from surprise import accuracy       # RMSE, MAE, cross-validation(k=fold)
from surprise.model_selection import train_test_split   # 훈련/검증 테스트 데이터 분류

# 실습
1. 데이터셋을 만들어주자 (사용자, 아이템, 평점), 훈련/검증 테스트 데이터 분류
2. SVD() 모델 선정
3. 훈련용 데이터로 fit()
4. 검증용 데이터로 예측
5. 정확도 계산
6. 사용자, 아이템정보 주고 예측 실행

In [4]:
# 1. 데이터셋을 만들어주자 (사용자, 아이템, 평점), 훈련/검증 테스트 데이터 분류
data = Dataset.load_builtin('ml-100k')
data

In [5]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [6]:
trainset

In [7]:
testset[:5]

[('120', '282', 4.0),
 ('882', '291', 4.0),
 ('535', '507', 5.0),
 ('697', '244', 5.0),
 ('751', '385', 4.0)]

In [8]:
# 2. SVD() 모델 선정
algo = SVD()

In [9]:
# 3. 훈련용 데이터로 fit()
algo.fit(trainset)

In [10]:
# 4. 검증용 데이터로 예측
algo.test(testset)[:5]

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.722759784777322, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.006061288252456, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9570184239145054, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6530412311190106, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.7767758471291466, details={'was_impossible': False})]

In [11]:
prediction = algo.test(testset)[:5]
prediction

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.722759784777322, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.006061288252456, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9570184239145054, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6530412311190106, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.7767758471291466, details={'was_impossible': False})]

In [12]:
[(pred.uid, pred.iid, pred.est) for pred in prediction]

[('120', '282', 3.722759784777322),
 ('882', '291', 4.006061288252456),
 ('535', '507', 3.9570184239145054),
 ('697', '244', 3.6530412311190106),
 ('751', '385', 3.7767758471291466)]

In [13]:
# 5. 정확도 계산

In [14]:
# 6. 사용자, 아이템정보 주고 예측 실행
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.129590526467769, details={'was_impossible': False})

# Surprise 주요 모듈 소개

In [15]:
import pandas as pd
from surprise import Reader

1. CSV를 SVD하기 위한 DataSet를 주기 위해서는
2. (1) df <---csv, (2) DataSet <---df

In [16]:
ratings = pd.read_csv('/content/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index와 header를 제거한 새로운 파일 형성
ratings.to_csv('/content/ratings_noh.csv', index=False, header=False)

In [17]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [18]:
reader = Reader(rating_scale=(0.5, 5))

In [19]:
# SVD에서 사용할 수 있는 데이터 셋으로 만들어주자
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data

In [20]:
# 데이터 분류, 객체생성, 훈련시키고, 검증해보고, 검증결과 프린트
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [21]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [22]:
algo = SVD(n_factors=50, n_epochs= 40,  random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 1.0388


1.0387802469854106

In [23]:
from surprise.model_selection import cross_validate

In [24]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8793  0.8741  0.8801  0.8695  0.8811  0.8768  0.0044  
MAE (testset)     0.6736  0.6699  0.6739  0.6676  0.6751  0.6720  0.0028  
Fit time          6.36    6.34    6.38    6.31    6.39    6.36    0.03    
Test time         0.28    0.16    0.31    0.28    0.16    0.24    0.06    


{'fit_time': (6.358381748199463,
  6.341940641403198,
  6.384150505065918,
  6.311557292938232,
  6.391209363937378),
 'test_mae': array([0.67362559, 0.66989635, 0.6739028 , 0.66763514, 0.67509143]),
 'test_rmse': array([0.87926321, 0.87406516, 0.88011601, 0.8695367 , 0.88106018]),
 'test_time': (0.2774498462677002,
  0.16372466087341309,
  0.31042981147766113,
  0.2795257568359375,
  0.15609979629516602)}

# Movies.csv

In [25]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [26]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


- 내가 안 본 영화 리스트를 구해서, 이 중에서 추천하려고 함

In [27]:
# def get_unseen_surprise(ratings, movies, userId):
#     #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
#     seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
#     # 모든 영화들의 movieId를 리스트로 생성. 
#     total_movies = movies['movieId'].tolist()
    
#     # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
#     unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
#     print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
#           '전체 영화수:',len(total_movies))
    
#     return unseen_movies

# unseen_movies = get_unseen_surprise(ratings, movies, 9)

In [28]:
# 수업 
def get_unseen_surprise(ratings, movies, userId):
    ## 전체 영화id 리스트
    total_movies = movies['movieId'].tolist()
    ## 내가 본 영화id 리스트
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    ## 추천 대상이 되는 영화 리스트 - 내가 본 영화 리스트
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

    print('전체 영화 리스트 개수 >>', len(total_movies))
    print('내가 본 영화 리스트 개수 >>', len(seen_movies))
    print('내가 안 본 영화 리스트 개수 >>', len(unseen_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

전체 영화 리스트 개수 >> 9742
내가 본 영화 리스트 개수 >> 46
내가 안 본 영화 리스트 개수 >> 9696


- 안 본 영화중에서 평점 예측이 높게 나온 3개를 리스트업하는 함수

In [29]:
# def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
#     # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
#     predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
#     # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
#     # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
#     # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
#     # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
#     def sortkey_est(pred):
#         return pred.est
    
#     # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
#     predictions.sort(key=sortkey_est, reverse=True)
#     top_predictions= predictions[:top_n]
    
#     # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
#     top_movie_ids = [ int(pred.iid) for pred in top_predictions]
#     top_movie_rating = [ pred.est for pred in top_predictions]
#     top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
#     top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
#     return top_movie_preds

# unseen_movies = get_unseen_surprise(ratings, movies, 9)
# top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
# print('##### Top-10 추천 영화 리스트 #####')

# for top_movie in top_movie_preds:
#     print(top_movie[1], ":", top_movie[2])

In [30]:
# 수업
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=5):
  # 안본 영화리스트를 하나씩 꺼낸 다음 평점을 예측하세요
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
  # pred = algo.predict(uid, iid, verbose=True)

  def sortkey_est(one):
    return one.est

  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [ int(pred.iid) for pred in top_predictions]
  top_movie_rating = [ pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  print(list(zip(top_movie_ids, top_movie_titles, top_movie_rating)))
  top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
  
  # print(len(predictions))
  # print(predictions[:5])
  return top_movie_preds

In [31]:
top_predictions = recomm_movie_by_surprise(algo, 9, unseen_movies)
top_predictions

[(1, 'Toy Story (1995)', 3.501196246389567), (2, 'Jumanji (1995)', 3.501196246389567), (3, 'Grumpier Old Men (1995)', 3.501196246389567), (4, 'Waiting to Exhale (1995)', 3.501196246389567), (5, 'Father of the Bride Part II (1995)', 3.501196246389567)]


[(1, 'Toy Story (1995)', 3.501196246389567),
 (2, 'Jumanji (1995)', 3.501196246389567),
 (3, 'Grumpier Old Men (1995)', 3.501196246389567),
 (4, 'Waiting to Exhale (1995)', 3.501196246389567),
 (5, 'Father of the Bride Part II (1995)', 3.501196246389567)]